In [2]:

import csv
import requests
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import numpy as np
import itertools

import yahoo_fin.stock_info as si
import yfinance as yf

import smtplib
from smtplib import SMTP
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.utils import COMMASPACE

In [3]:
### DIVIDEND DATA HANDLING MODULE ###

# Date and Time handling
date = '4/16/2021'
date_today = dt.date.today()
format_today = date_today.strftime('%m/%d/%Y')
csvformat_today = date_today.strftime('%m-%d-%Y')

# URL and BS4 handling
url = ('https://eresearch.fidelity.com/eresearch/conferenceCalls.jhtml?tab=dividends&begindate={}'.format(format_today))
geturl = requests.get(url)
print(geturl.status_code)
coverpage = geturl.content
yahoosoup = BeautifulSoup(coverpage)

# Ticker information
ticker_list = []
def generate_report():
    # Ticker information
    html_tickerinfo = yahoosoup.find_all('td', class_='lft-rt-border center blue-links')
    try:
        for symbol in html_tickerinfo:
            # print(html_tickerinfo)
            ticker_list.append(symbol.a.contents[0])
            tickers_df = pd.DataFrame(ticker_list, columns=['Ticker'])
    except:
        for symbol in html_tickerinfo:
            # print(html_tickerinfo)
            print((symbol.a))
            ticker_list.append(symbol.a)
            tickers_df = pd.DataFrame(ticker_list, columns=['Ticker'])
        ticker_col = pd.Series(tickers_df['Ticker'])
        #print(ticker_col)
    

    # Ex-dividend date information
    ex_div_dates_df = pd.DataFrame()
    len_tickers = len(tickers_df)
    
    ex_date_list = list(itertools.repeat(format_today, len_tickers))
    ex_div_dates_df.insert(0, 'Ex-Date', ex_date_list)
    
    #ex_div_dates.append()

# Dividend Amount information
    dividend_list = []
    html_divs = yahoosoup.find_all('td', class_='right')
    for div in html_divs:
        dividend_list.append(div.contents[0])
        dividend_df = pd.DataFrame(dividend_list, columns=['Dividends'])
        
# Record Date information



# Pay Date information
    pay_days = []
    html_pay_dates = yahoosoup.find_all('td', class_='lft-rt-border')
    #for dates in html_pay_dates:
        #print(dates.td.con)



# Join and merge all information to one dataframe
    frames = [tickers_df, dividend_df, ex_div_dates_df]
    
    masterframe = pd.concat(frames, axis=1)
    master_df = pd.DataFrame(masterframe)
    
    #return masterframe
    
    # Write the master dataframe to csv file
    # masterframe.to_csv(('CSVfiles/{}_Dividend_list').format(format_today))
    masterframe.to_csv('CSVfiles/dividends_for_{}.csv'.format(csvformat_today))
    return master_df

# Additional Financial information module
    #ticks = pd.DataFrame(ticker_col)
    
    #print(ticks.values)
    

generate_report()

200


UnboundLocalError: local variable 'tickers_df' referenced before assignment

In [4]:
### EARNINGS INFORMATION

def earnings_this_week(startdate, enddate):
    
    # Start and End Dates
    # startdate = '05/10/2021'
    # enddate = '05/16/2021'
    
    # Earnings Data in DataFrame Format
    earnings_this_week = si.get_earnings_in_date_range(startdate, enddate)
    earnings_thiswk = pd.DataFrame(earnings_this_week)
    
    # Retrive Market Cap and Stock Price Information

    
    # Retreives Week Number
    today = dt.date.today()
    weeknumber = today.isocalendar()[1]
    
    
    # Write to CSV file in 'CSVfiles' folder
    earnings_thiswk.to_csv('CSVfiles/earnings_for_week_{}.csv'.format(weeknumber))
    dropcols = ['startdatetimetype', 'timeZoneShortName', 'gmtOffsetMilliSeconds' ,'quoteType','epsactual','epssurprisepct']
    editearningsdf = earnings_thiswk.drop(columns=dropcols)
    
    
    
    return editearningsdf

earnings_this_week('05/9/2021','05/16/2021')

,ticker,companyshortname,startdatetime,epsestimate
0,FPLSF,5N Plus Inc,2021-05-10T20:00:00.000Z,NaN
1,BWLA,Bowl America Incorporated,2021-05-11T00:00:00.000Z,NaN
2,AIRI,Air Industries Group,2021-05-11T00:00:00.000Z,0.00
3,MTAGF,Ceconomy AG,2021-05-11T00:00:00.000Z,NaN
4,TCMFF,Telecom Argentina S.A.,2021-05-11T00:00:00.000Z,NaN
...,...,...,...,...
1013,TSUKF,Toyo Suisan Kaisha Ltd,2021-05-14T10:59:00.000Z,NaN
1014,SAWLF,Shawcor Ltd,2021-05-14T10:59:00.000Z,NaN
1015,NBLX,Noble Midstream Partners LP,2021-05-14T12:30:00.000Z,0.52
1016,SOHU,Sohu.com Ltd,2021-05-14T12:30:00.000Z,0.48


In [ ]:
### DIVIDEND EMAIL MODULE ###
    
def send_div_report():
    
    # Credentials 
    sender_email = 'robotydividend@gmail.com'
    pword = input(str("Please enter your password..."))
    
    # Recipiants List
    recipiant = ['ryanhhogan@gmail.com', 'dominicchimienti@gmail.com']
    #recipiant = "<ryanhhogan@gmail.com>"

    # Subject, Messege of email
    initialmsg = 'Hello, here are compaines that are reporting earnings for week 19'
    

    ##########################################################
    
    # The target CSV file
    dividend_data = open('CSVfiles/dividends_for_{}.csv'.format(csvformat_today))
    
    # joining with space content of text
    joined_data = ' '.join([i for i in dividend_data])

    # replace the commas by space
    text_data = joined_data.replace(",","  |  ")

    # Assign our results to our message variable
    message = initialmsg + text_data
    
    

    # Sending the email 
    server = smtplib.SMTP('smtp.gmail.com',587)
    server.ehlo()
    server.starttls()
    server.login(sender_email, pword)
    #server.connect()
    print('Login success')

    server.sendmail(sender_email, recipiant, message)
    print('Email has been sent to ', recipiant)
    server.quit()

send_div_report()

In [ ]:
class person:
    def __init__(self, age, weight, height, first_name, last_name, phrase):
        self.age = age
        self.weight = weight
        self.height = height
        self.first_name = first_name
        self.last_name = last_name
        self.phrase = phrase

user = person(28,80,6.6,"jon","snow", "hello, I have something for you")

In [ ]:
user.phrase

In [42]:
#Earningsdata = earnings_this_week('05/9/2021','05/16/2021')
ticker = 'aapl'


def get_marketcap(ticker):
    statsinfo = si.get_stats_valuation(ticker)
    statsdf = pd.DataFrame(statsinfo)
    marketcaps = pd.DataFrame(statsdf.loc[0])
    marketcaparray = marketcaps.values[1]
    marketcap = marketcaparray.tolist()
    return marketcap

get_marketcap(ticker)

['2.10T']

In [5]:
Earningsdf = earnings_this_week('05/9/2021','05/16/2021')
tickerdf = Earningsdf['ticker']

mktcaplist = []

#for ticker in Earningsdf:
    #mktcaplist.append(get_marketcap(ticker))
    

In [60]:
tickerdf = Earningsdf['ticker']
mktcaplist = []
pricelist = []
def gets_marketCap(tickerdf):
    for ticker in tickerdf:
        stocks = yf.Ticker(ticker)
        marketcap = stocks.info['marketCap']
        price = stocks.info['previousClose']
        mktcaplist.append(marketcap)
        pricelist.append(price)
    return marketcap
    
gets_marketCap(tickerdf)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [62]:
mktcaps = pd.DataFrame(mktcaplist)
prices = pd.DataFrame(pricelist)
prices

,0
0,24.03
1,25.19
2,6.29
3,22.75
4,23.00
...,...
62,12.45
63,3.39
64,4.96
65,5.84


In [59]:
ticker = 'aapl'
stocks = yf.Ticker(ticker)
marketcap = stocks.info['marketCap']
price = stocks.info['previousClose']

{'zip': '95014',
 'sector': 'Technology',
 'fullTimeEmployees': 100000,
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand r

In [78]:
### Trying to automate earnings module

# Grabs companies reporting earnings between input dates
def get_earnings_fordates(startdate, enddate):
    
    # Start and End Dates
    # startdate = '05/10/2021'
    # enddate = '05/16/2021'
    
    # Earnings Data in DataFrame Format
    earnings_this_week = si.get_earnings_in_date_range(startdate, enddate)
    earnings_thiswk = pd.DataFrame(earnings_this_week)
    
    # Retrive Market Cap and Stock Price Information

    
    # Retreives Week Number
    today = dt.date.today()
    weeknumber = today.isocalendar()[1]
    
    
    # Write to CSV file in 'CSVfiles' folder
    earnings_thiswk.to_csv('CSVfiles/earnings_for_week_{}.csv'.format(weeknumber))
    dropcols = ['startdatetimetype', 'timeZoneShortName', 'gmtOffsetMilliSeconds' ,'quoteType','epsactual','epssurprisepct']
    editearningsdf = earnings_thiswk.drop(columns=dropcols)
    
    return editearningsdf

# Calculates the input dates for the week (release every sunday)
def this_weeks_earnings():
    today = dt.date.today()
    one_week = dt.timedelta(days=7)
    one_week_ahead = today + one_week
    today = str(today)
    one_week = str(one_week_ahead)
    
    # Call earnings for week function
    earningsDF = get_earnings_fordates(today, one_week)
    return earningsDF
    
    
this_weeks_earnings()
#earnings_this_week(today,one_week)

,ticker,companyshortname,startdatetime,epsestimate
0,HUTMF,Hut 8 Mining Corp,2021-05-13T10:00:00.000Z,NaN
1,MASS,908 Devices Inc.,2021-05-13T08:30:00.000Z,-0.22
2,MASS,908 Devices Inc.,2021-05-13T12:30:00.000Z,-0.22
3,CRLFF,Cardinal Energy Ltd (Alberta),2021-05-13T20:00:00.000Z,NaN
4,AZEK,Azek Company Inc,2021-05-13T12:30:00.000Z,0.23
...,...,...,...,...
450,TLGT,Teligent Inc (NEW JERSEY),2021-05-20T12:30:00.000Z,NaN
451,ROIUF,Route1 Inc,2021-05-20T09:00:00.000Z,NaN
452,SVM,Silvercorp Metals Inc,2021-05-20T20:00:00.000Z,0.04
453,MAXN,Maxeon Solar Technologies Ltd,2021-05-20T12:30:00.000Z,-1.48


In [ ]:
### OPTIONS CHAIN ANALYSIS


## STEP 1: Analyze the Chain for Oppertunities
# This step is simply identifying unusual options trades and validating them by comparing them with other data

# 1.1 Compare call/put option volumes > average daily vol ((3 mo. is most important), 6 mo., 1y)

# 1.2 Compare Open Interest: Ensure current volume exceeds prior days open int. (this indicates new activity/positions)

## STEP 2: Determine Magnitude of MM prediction with staddles

# 2.1 Look at the total cost of ATM straddle (long call, Long put)

# 2.2 Take total cost and divide by 100 share block cost (share price x 100 = block cost)
#     the quotient is the percentage move that is anticipated

# 2.3 Look for data that shows what TYPES of trades are being place (single leg, multi-leg, etc.)